In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 1
setup_one_gpu(GPU)

Picking GPU 1


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter
from tf_lab.point_clouds.ae_templates import conv_architecture_ala_nips_17, default_train_params_ala_nips_17
from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry, decoder_with_fc_only
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_points = 2048
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
top_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_points))

In [5]:
train_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/train.txt')
val_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/val.txt')
test_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/test.txt')

In [6]:
do_training = True
load_pretrained_model = False
load_epoch = None
random_seed = 42
splitter = Data_Splitter(top_pclouds_path, data_file_ending='.ply', random_seed=random_seed)

In [7]:
bneck_size = 32
loss = 'chamfer'

In [20]:
tr_files = splitter.load_splits(train_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(tr_files, n_threads=20, loader=snc_loader, verbose=True)
train_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)

val_files = splitter.load_splits(val_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(val_files, n_threads=20, loader=snc_loader, verbose=True)
val_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)

test_files = splitter.load_splits(test_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(test_files, n_threads=20, loader=snc_loader, verbose=True)
val_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)

5761 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.
679 pclouds were loaded. They belong in 1 shape-classes.


In [43]:
b_norm = False

encoder_args = {'n_filters': [64, 128, 128, 256],
                'filter_sizes': [16, 8, 8, 5],
                'strides': [2, 2, 1, 1],
                'b_norm': b_norm,
                }

decoder_args = {'layer_sizes': [bneck_size, 512, 1024, n_pc_points*3], 'b_norm': b_norm}

In [45]:
#OUT DIR to save models
v = 1
v = str(v)
experiment_id = 'neighborhood_size_v_' + v
experiment_name = '_'.join(['chair', 'exp', experiment_id, str(n_pc_points),
                            'pts_bneck', str(bneck_size), loss
                           ])

train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/testing_ae_settings', experiment_name)
create_dir(train_dir)

'/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/testing_ae_settings/chair_exp_neighborhood_size_v_1_4096_pts_bneck_32_chamfer'

In [46]:
if load_pretrained_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
    if conf.train_dir != train_dir:
        conf.train_dir = train_dir
    conf.save(osp.join(train_dir, 'configuration'))
else:
    conf = Conf(
                n_input = [n_pc_points, 3],
                loss = loss,
                training_epochs = 100,
                batch_size = 50,
                denoising = False,
                learning_rate = 0.0005,
                train_dir = train_dir,             
                loss_display_step = 1,
                saver_step = 10,  # save every 10 epochs
                z_rotate = False,
                encoder = encoder_with_convs_and_symmetry, 
                decoder = decoder_with_fc_only,
                encoder_args = encoder_args,
                decoder_args = decoder_args
               )
    
    conf.experiment_name = experiment_name
    conf.held_out_step = 5
    conf.save(osp.join(conf.train_dir, 'configuration'))

In [47]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)
print ae.trainable_parameters()

[64, 128, 128, 256]
[16, 8, 8, 5]
[2, None, 2, None]
Tensor("chair_exp_neighborhood_size_v_1_4096_pts_bneck_32_chamfer_1/AvgPool1D/Squeeze:0", shape=(?, 1024, 64), dtype=float32) 65536
Tensor("chair_exp_neighborhood_size_v_1_4096_pts_bneck_32_chamfer_1/Relu_1:0", shape=(?, 512, 128), dtype=float32) 65536
Tensor("chair_exp_neighborhood_size_v_1_4096_pts_bneck_32_chamfer_1/AvgPool1D_1/Squeeze:0", shape=(?, 256, 128), dtype=float32) 32768
Tensor("chair_exp_neighborhood_size_v_1_4096_pts_bneck_32_chamfer_1/Relu_3:0", shape=(?, 256, 256), dtype=float32) 65536
Tensor("chair_exp_neighborhood_size_v_1_4096_pts_bneck_32_chamfer_1/Max:0", shape=(?, 256), dtype=float32) 256
13514016


In [48]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)    
    train_stats = ae.train(train_data, conf, log_file=fout, held_out_data=val_data)
    fout.close()

('Epoch:', '0001', 'training time (minutes)=', '0.0671', 'loss=', '0.006659985')
('Epoch:', '0002', 'training time (minutes)=', '0.0641', 'loss=', '0.003197135')
('Epoch:', '0003', 'training time (minutes)=', '0.0649', 'loss=', '0.003309099')
('Epoch:', '0004', 'training time (minutes)=', '0.0651', 'loss=', '0.003315851')
('Epoch:', '0005', 'training time (minutes)=', '0.0660', 'loss=', '0.003068441')
('Held Out Data :', 'forward time (minutes)=', '0.0034', 'loss=', '0.002471049')
('Epoch:', '0006', 'training time (minutes)=', '0.0649', 'loss=', '0.002694160')
('Epoch:', '0007', 'training time (minutes)=', '0.0647', 'loss=', '0.002723398')
('Epoch:', '0008', 'training time (minutes)=', '0.0648', 'loss=', '0.002392891')
('Epoch:', '0009', 'training time (minutes)=', '0.0662', 'loss=', '0.002083382')
('Epoch:', '0010', 'training time (minutes)=', '0.0656', 'loss=', '0.001965301')
('Held Out Data :', 'forward time (minutes)=', '0.0031', 'loss=', '0.001821794')
('Epoch:', '0011', 'training

In [ ]:
lala = ae.reconstruct("give_input")
Point_Clouds("lala=2048x3 points").plot()  # to see.